# Model selection

In [1]:
from predict_model import clint_scorer, trump_scorer
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import re
import seaborn as sns
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, make_scorer
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import sys
import os
sys.path.append('../src/models')

In [2]:
with open('../data/processed/predictors.pickle', 'rb') as file:
    X = pickle.load(file)
with open('../data/processed/target.pickle', 'rb') as file:
    y = pickle.load(file)

In [3]:
clinton_metric = make_scorer(clint_scorer)
trump_metric = make_scorer(trump_scorer)

## Logistic regression

In [10]:
lr = LogisticRegression(
    C=2, penalty='l2', solver='lbfgs', multi_class='auto', max_iter=1000)

In [13]:
clinton_lr_scores = cross_val_score(lr, X, y, cv=5, scoring=clinton_metric, n_jobs=-1)
clinton_lr_scores

array([92.7, 89.1, 86.9, 90.3, 89.3])

In [19]:
print(clinton_rf_scores.mean())

89.41999999999999


In [14]:
trump_lr_scores = cross_val_score(lr, X, y, cv=5, scoring=trump_metric, n_jobs=-1)
trump_lr_scores

array([93.2, 83.6, 88.6, 85.3, 86.9])

In [20]:
trump_lr_scores.mean()

87.52000000000001

## Random forest

In [16]:
rf = RandomForestClassifier(n_estimators=100)

In [17]:
clinton_rf_scores = cross_val_score(rf, X, y, cv=5, scoring=clinton_metric, n_jobs=-1)
clinton_rf_scores

array([93.9, 90.1, 86.5, 88.4, 88.2])

In [18]:
print(clinton_rf_scores.mean())

89.41999999999999


In [ ]:
trump_rf_scores = cross_val_score(rf, X, y, cv=5, scoring=trump_)

## K Nearest Neighbors

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred_test = knn.predict(X_test)

In [ ]:
knn_metrics = create_perf_metrics(y_test, y_pred_test)

## Support vector machines

In [ ]:
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)
y_pred_test = svm.predict(X_test)

In [ ]:
# call performance metric function
svm_metrics = create_perf_metrics(y_test, y_pred_test)

## K-means clustering

In [ ]:
kmeans = KMeans(n_clusters=2)
kmeans.fit(X)

labels = kmeans.labels_
plt.figure(figsize=(5,5))
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], c='black', s=300)

y_pred = kmeans.predict(X)
metrics.silhouette_score(X, labels, metric='euclidean')
metrics.calinski_harabasz_score(X, labels)

y_pred_df = pd.DataFrame(y_pred, columns=['predicted'])
df = pd.DataFrame(y, columns=['presvote16post_2016']).join(y_pred_df)
num_cluster_trump = df.loc[(df.presvote16post_2016 == 2) & (df.predicted == 0)].count()
num_cluster_clinton = df.loc[(df.presvote16post_2016 == 1)].count()

# Parameter tuning

gridsearchcv

# Test 

Which model does better with test data? 
Stick with one metric
Whatever model is best, variation between train and test,etc. 